In [1]:
import pandas as pd
import numpy as np
import os
import scipy
import sqlite3
import tensorflow as tf
import string
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression

In [2]:
# create our data set
curr_path = os.path.realpath('')
conn = sqlite3.connect(curr_path+"/lending-club-loan-data/database.sqlite")

###@@@@@ 
#         add loan_amnt and annual_inc back later 
###@@@@

df_tf = pd.read_sql_query("select loan_status, term, int_rate, emp_length, home_ownership, verification_status, dti from loan where id in (select id from loan order by random()) AND emp_length != 'n/a' AND int_rate != 'NULL' AND loan_status =='Fully Paid';", conn)
df_tf_default = pd.read_sql_query("select loan_status, term, int_rate, emp_length, home_ownership, verification_status, dti from loan where id in (select id from loan order by random()) AND emp_length != 'n/a' AND int_rate != 'NULL' AND loan_status == 'Charged Off' OR loan_status == 'Default';", conn)

df_tf_test = pd.read_sql_query("select loan_status, term, int_rate, emp_length, home_ownership, verification_status, dti from loan where id in (select id from loan order by random()) AND emp_length != 'n/a' AND int_rate != 'NULL' AND loan_status == 'Charged Off' OR loan_status == 'Default' OR loan_status == 'Fully Paid';", conn)

conn.close()

# df_desc = pd.read_sql_query("select loan_amnt, loan_status, sub_grade from loan where desc != 'NULL' AND loan_status !='Current' AND loan_status !='Late (31-120 days)' AND loan_status != 'Late (16-30 days)'", conn)
df_tf = df_tf.sample(frac=(25000/float(len(df_tf)))).reset_index(drop=True)
df_tf_default = df_tf_default.sample(frac=25000/float(len(df_tf_default))).reset_index(drop=True)
df_tf_test = df_tf_test.sample(frac=10000/float(len(df_tf_default))).reset_index(drop=True)

In [3]:
print (len(df_tf))
print(len(df_tf_default))

# this set is split between half paid and half default/charged-offs
print (len(df_tf))
df_tf = df_tf.append(df_tf_default, ignore_index=True)
print (len(df_tf))

#results = [0]*40000 + [1]*40000
# df_tf = split_data

25000
25000
25000
50000


In [4]:
data = []
results = []
for i in range(len(df_tf)):  # data set of 10000 for faster run time. Replace with range(len(df_tf)) later
    
    #print (i)
    loan_status = 0 if df_tf.iloc[i][0] == 'Fully Paid' else 1 # bad if not fully paid
    results.append(loan_status)     # add  to results 
    
    # 0 if term is 36 months, 1 if 60 months
    term = 0 if df_tf.iloc[i][1] == ' 36 months' else 1
    
    # change interest rate from str to float
    if type(df_tf.iloc[i][2]) == str:     # condition to prevent calling replace on NoneTypes
        int_rate = float(df_tf.iloc[i][2].replace('%', ''))
    
    # get employment length
    txt = ''.join([c for c in df_tf.iloc[i][3].lower() if not c in set(string.punctuation)])
    emp_length_list = ([int(s) for s in txt.split() if s.isdigit()])
    emp_length = 0 if emp_length_list == [] else emp_length_list[0]

    # 0 if renting, 1 if owner
    ownership = 0 if df_tf.iloc[i][4] == 'RENT' else 1
    
    # verification sttus, 0 for not verified, 1 for source verified and 2 for verified
    #verification = 0 if df_tf.iloc[i][5] == 'Not Verified' else 1 if df_tf.iloc[i][5] == 'Source Verified' else 2
    verification = 0 if df_tf.iloc[i][5] == 'Not Verified' else 1

    # append in feature vector
   # data.append([term, int_rate, emp_length, ownership, verification, df_tf.iloc[i][6], df_tf.iloc[i][7], df_tf.iloc[i][8]])
    data.append([term, int_rate, emp_length, ownership, verification, df_tf.iloc[i][6]])


In [5]:
data_test = []
results_test = []
for i in range(len(df_tf_test)):  # data set of 10000 for faster run time. Replace with range(len(df_tf)) later
    
    #print (i)
    loan_status = 0 if df_tf_test.iloc[i][0] == 'Fully Paid' else 1 # bad if not fully paid
    results_test.append(loan_status)     # add  to results 
    
    # 0 if term is 36 months, 1 if 60 months
    term = 0 if df_tf_test.iloc[i][1] == ' 36 months' else 1
    
    # change interest rate from str to float
    if type(df_tf_test.iloc[i][2]) == str:     # condition to prevent calling replace on NoneTypes
        int_rate = float(df_tf_test.iloc[i][2].replace('%', ''))
    
    # get employment length
    txt = ''.join([c for c in df_tf_test.iloc[i][3].lower() if not c in set(string.punctuation)])
    emp_length_list = ([int(s) for s in txt.split() if s.isdigit()])
    emp_length = 0 if emp_length_list == [] else emp_length_list[0]

    # 0 if renting, 1 if owner
    ownership = 0 if df_tf_test.iloc[i][4] == 'RENT' else 1
    
    # verification sttus, 0 for not verified, 1 for source verified and 2 for verified
    #verification = 0 if df_tf.iloc[i][5] == 'Not Verified' else 1 if df_tf.iloc[i][5] == 'Source Verified' else 2
    verification = 0 if df_tf_test.iloc[i][5] == 'Not Verified' else 1

    # append in feature vector
   # data.append([term, int_rate, emp_length, ownership, verification, df_tf.iloc[i][6], df_tf.iloc[i][7], df_tf.iloc[i][8]])
    data_test.append([term, int_rate, emp_length, ownership, verification, df_tf_test.iloc[i][6]])


In [6]:
X_train = data
y_train = results

X_val = data_test[:int(len(data)*0.3)]
y_val = results_test[:int(len(results)*.3)]

X_test1 = data_test[int(len(data_test)*.3) : int(len(data_test)*0.5)]
y_test1 = results_test[int(len(results_test)*.3) : int(len(data_test)*0.5)]

X_test2 = data_test[int(len(data_test)*.5) : int(len(data_test)*0.7)]
y_test2 = results_test[int(len(results_test)*.5) : int(len(data_test)*0.7)]

X_test3 = data_test[int(len(data_test)*.7) : int(len(data_test)*0.9)]
y_test3 = results_test[int(len(results_test)*.7) : int(len(data_test)*0.9)]

X_test4 = data_test[int(len(data_test)*.9) : int(len(data_test)*1)]
y_test4 = results_test[int(len(results_test)*.9) : int(len(data_test)*1)]

In [30]:
### normalize, scale and train on Logistic Regressor and Neural Net

X_train_norm = preprocessing.normalize(X_train)
scaler = preprocessing.StandardScaler().fit(X_train_norm)
X_train_norm_std = scaler.transform(X_train_norm)

X_train_arr = np.array(X_train_norm_std)
y_train_arr = np.array(y_train)

# training with DNNClassifier
feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train_arr)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[512, 256], n_classes=2, dropout = .9, feature_columns=feature_cols)
dnn_clf.fit(X_train_arr, y_train_arr, batch_size=6, steps=100)

# training with Logistic Regression
log_clf = LogisticRegression()
log_clf.fit(X_train_norm, y_train)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a229037f0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': None}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving 

/Users/noellin/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/cv/j7y0jf0x0gbc39xntlnng44c0000gn/T/tmp9y8r2zmx/model.ckpt.
INFO:tensorflow:loss = 0.531105, step = 1
INFO:tensorflow:Saving checkpoints for 100 into /var/folders/cv/j7y0jf0x0gbc39xntlnng44c0000gn/T/tmp9y8r2zmx/model.ckpt.
INFO:tensorflow:Loss for final step: 0.754116.


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [16]:
# Function to test with logistic regressor and neural net
# params: X, y, neural net classifier and logistic regressor
def runTests(X, y, dnn_clf, log_clf, setName):
    X_norm = preprocessing.normalize(X)   # normalize, scale and turn X into array 
    X_arr = np.array(scaler.transform(X_norm))
    y_arr = np.array(y)
    y_pred = dnn_clf.predict_classes(X_arr, as_iterable=False)
    print ('Running' + setName + 'set')
    print('###')
    print('Neural Net accuracy: ' + str(accuracy_score(y_arr, y_pred)))
    y_pred = log_clf.predict(X_norm)
    print('Log. Regression accuracy: ' + str(accuracy_score(y, y_pred)))
    print('###')

In [31]:
runTests(X_train, y_train, dnn_clf, log_clf, 'training')
runTests(X_val, y_val, dnn_clf, log_clf, 'validation')
runTests(X_test1, y_test1, dnn_clf, log_clf, 'test 1')
runTests(X_test2, y_test2, dnn_clf, log_clf, 'test 2')

Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from /var/folders/cv/j7y0jf0x0gbc39xntlnng44c0000gn/T/tmp9y8r2zmx/model.ckpt-100


/Users/noellin/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


Runningtrainingset
###
Neural Net accuracy: 0.58878
Log. Regression accuracy: 0.60116
###
Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from /var/folders/cv/j7y0jf0x0gbc39xntlnng44